# Classification

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split


In [ ]:
# import des données préparées
data = pd.read_csv("20240125_dataset_pickle/data.csv", index_col=0)
X = data.titre.values
y = data.domaine.values

# import du graphe
G= nx.read_gexf("data/coauteur.gexf")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data,
                                                          y,
                                                          train_size = 0.8,
                                                          test_size=0.2, 
                                                          stratify = y,
                                                          random_state=19032024
                                                         )

In [ ]:
domaine_code=[np.where(np.array(list(dict.fromkeys(y)))==e)[0][0]for e in y]

# Affichage du graphe
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color=domaine_code, node_size=10, font_size=10)
nx.draw_networkx_labels(G, pos, labels=domaine_code, font_size=12, font_color='red')
plt.title("Graphe des articles scientifiques avec des étiquettes")
plt.show()

In [ ]:
# Création de la matrice d'adjacence et des étiquettes
adj_matrix = nx.to_numpy_matrix(G)
y = np.array([domaine_code.get(node, None) for node in G.nodes()])

y2=y_test
y2=-1
labels=pd.concat(y_train,y2)
# Fit du modèle
label_prop_model = LabelPropagation()
label_prop_model.fit(adj_matrix, labels)

# Prédiction des étiquettes pour tous les nœuds
predicted_labels = label_prop_model.transduction_

# Évaluation des résultats
print("Classification Report:")
print(classification_report(labels, predicted_labels))

# Matrice de confusion
print(classification_report(labels, predicted_labels))
conf_matrix = confusion_matrix(labels, predicted_labels)
plt.figure(figsize=(6, 4))
plt.imshow(conf_matrix, cmap=plt.cm.Blues)
plt.colorbar()
plt.title('Matrice de confusion')
plt.xlabel('Classe prédite')
plt.ylabel('Classe réelle')
plt.show()